# Tutorial for HPS multiple A1-LCD WTARO 

In [2]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import time
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb
from openabc.utils.insert import insert_molecules

# set simulation platform
#platform_name = 'CPU'
platform_name = 'CUDA'
#platform_name = 'OpenCL'

Parse a single A1LCD WT. The sequence length is 137.

In [6]:
#sequence = 'GSMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF' # WT
sequence = 'GSMASASSSQRGRSGSGNSGGGRGGGFGGNDNFGRGGNSSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNSGGGGSSNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYSAKPRNQGGYGGSSSSSSSGSGRRF' # ARO-
ca_pdb = 'init_A1LCD_WT_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)

Parse molecule with default settings.


Prepare the initial configuration.

In [7]:
# insert molecules into the simulation box randomly
n_mol = 700
if not os.path.exists('start.pdb'):
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])
else:
    # delete and create a new start.pdb
    # otherwise, there will be an error for init_coord in the next step
    os.remove('start.pdb')
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])


Successfully inserted 700 molecules.


Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [8]:
protein = HPSModel()
for i in range(n_mol):
    protein.append_mol(protein_parser)
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
print('size of init_coord:', len(init_coord))
protein.create_system(top, box_a=100, box_b=100, box_c=100)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 274*unit.kelvin
friction_coeff = 0.01/unit.picosecond # use smaller friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 1000
output_dcd = 'output_multi_A1LCD_WT.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)

# run simulation
start = time.time()
totaL_steps = 100000
protein.simulation.step(totaL_steps)
end = time.time()
print('Total time for %d steps: %.2f s' % (totaL_steps, end-start))

size of init_coord: 95900
Add protein bonds.
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CUDA
Use precision: mixed
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,9.999999999999831,47204.85861786125,280510.46366298123,327715.32228084246,234.53601586998673,0
2000,20.000000000000327,35631.92022655329,299032.5787242653,334664.49895081855,250.02243664457237,1.7e+03
3000,30.00000000000189,32216.580350166998,306430.20659059804,338646.78694076504,256.2076254036733,1.68e+03
4000,40.00000000000061,30775.99096463786,311689.5255294422,342465.51649408,260.604964789872,1.68e+03
5000,49.99999999999862,30465.4654731287,314795.9691761922,345261.4346493209,263.2022758025147,1.68e+03
6000,59.99999999999663,31199.28526903711,316193.2052120809,347392.

In [5]:
# visualize trajectory
traj = mdtraj.load_dcd('output_multi_A1LCD_WT.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=99)